# Images
本部分的主要任务是实现对image的处理，形成对pytorch来说比较适合的tensor等多种格式

In [5]:
import imageio
import torch

In [3]:
img_arr = imageio.imread('./data/image-dog/bobby.jpg')
img_arr.shape

(720, 1280, 3)

pytroch 中image data的格式应该是：CxHxW，也就是先通道，再高度，再宽度的形式  
tensorflow中是H W C

In [7]:
img = torch.from_numpy(img_arr)
out = torch.transpose(img, 0, 2)

当有很多张图片时，需要把batch放在第一个维度，形成 $N\cdot C\cdot H\cdot N$的张量

有一种方法，可以首先生成一个确定大小的tensor，然后再用图片去填充它

In [8]:
batch_size = 100
batch = torch.zeros(100, 3, 256, 256, dtype=torch.uint8)

In [10]:
import os

data_dir = './data/image-cats/'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)=='.png']
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(filename)
    batch[i] = torch.transpose(torch.from_numpy(img_arr), 0, 2)
batch.shape

torch.Size([100, 3, 256, 256])

从前面的学习得出，神经网络的数据最好是归一化的，比如在0-1之间，或者-1到1之间  
而且此处还需要转化成浮点数  
所以接下来有两项内容：
- 将数据转化为浮点数
- 将数据归一化、标准化 normalization
这里使用的归一化方法就是直接除以255（8-bit二进制数的最大数值）

In [11]:
batch = batch.float()
batch /= 255.0

此外，计算平均值和标准差，从而使每个通道的输出结果是0-均值和单位标准差的

In [12]:
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

还有其他的图像预处理操作，比如：
- geometric transformations 旋转、平移、裁剪